# Projeto 1 - Ciência dos Dados

Nome: Lucca Martins Pereira Lima

Nome: Christian Delfs

Nome: Vitor de Moraes Caixeta

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [8]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [9]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Vitor Caixeta\OneDrive\C_dados DP\Nova pasta\22-2b-cd-p1-grupo_luccampl\notebooks


Carregando a base de dados com as notícias classificadas manualmente:

In [10]:
filename = 'dados.xlsx'
dados = pd.read_excel(filename)

In [11]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Categoria,Titulo,Descrição,Data,Pagina,Target
0,tecnologia,Pesquisadores encontram fóssil do maior inseto...,Paleontologistas ingleses descobriram o fóssil...,25/12/2021 16:51,66.0,2.0
1,tecnologia,WhatsApp define “head” no Brasil de olho em el...,O WhatsApp anunciou em março que o Brasil seri...,04/04/2022 17:39,37.0,1.0
2,tecnologia,Os tropeços da vacina Sputnik V na América Latina,A Sputnik V foi a primeira vacina contra a cov...,04/08/2021 17:03,121.0,0.0
3,tecnologia,Variante britânica do coronavírus aponta uma m...,"A variante britânica do coronavírus, considera...",22/01/2021 14:08,238.0,0.0
4,tecnologia,Vacinação contra a covid-19 na Venezuela: espe...,Yaritza de Velázquez se preparava para visitar...,05/05/2021 17:07,169.0,0.0


In [12]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Categoria,Titulo,Descrição,Data,Pagina,Target
0,tecnologia,Eclipse lunar ‘quase total’ na próxima noite s...,A América do Norte e grande parte da América d...,18/11/2021 23:00,80,2
1,tecnologia,"Pandemia vai acabar em 1 ano, diz CEO da farma...","O CEO da Moderna, Stéphane Bancel, declarou qu...",24/09/2021 12:06,99,0
2,tecnologia,Moderna acredita que sua dose de reforço funci...,A farmacêutica americana Moderna disse nesta s...,21/12/2021 03:35,68,0
3,tecnologia,Mais alto e por mais tempo: helicóptero Ingenu...,"Ingenuity, o pequeno helicóptero da NASA, cons...",22/04/2021 15:01,177,1
4,tecnologia,Argentina pede que vacinas contra a covid-19 s...,"O presidente da Argentina, Alberto Fernández, ...",28/01/2021 19:25,233,0


___
## Classificador automático


Para o projeto, utilizamos uma base de dados referente ao site https://www.istoedinheiro.com.br/. 
A ideia do nosso classificador foi de diferenciar, associar e classificar 3 diferentes tipos de notícias, na seção de "tecnologia" do site.
Sendo os fatores de classificação: 'Notícias relacionadas à tecnologias e saúde', 'Notícias relacionadas à avanços da tecnologia' e também notícias que diferem desses 2 temas, sendo essas classificadas como "outras".

0 - Notícias relacionadas à saúde

1 - Notícias relacionadas ao avanço de tecnologias

2 - Notícias que não se relacionam à nenhum dos 2 (outras)

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [13]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;123456789%]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [14]:
# Removendo as colunas que não serão utilizadas na análise, visto que ela é baseada no título das notícias
train = train.drop(['Descrição', 'Data', 'Pagina', 'Categoria'], axis = 1)


In [15]:
filtro_0 = train.Target == 0
dados_target_0 = train.loc[filtro_0, 'Titulo'].reset_index(drop=True)

filtro_1 = train.Target == 1
dados_target_1 = train.loc[filtro_1, 'Titulo'].reset_index(drop=True)

filtro_2 = train.Target == 2
dados_target_2 = train.loc[filtro_2, 'Titulo'].reset_index(drop=True)


In [16]:
# Cria as listas que possuem as classificações de cada categoria
lista_dados_target_0 = []
lista_dados_target_1 = []
lista_dados_target_2 = []

lista_noticias_limpas = []
lista_noticias = []

# For criando as listas limpas
for cell in dados_target_0:
    
    clean_cell = cleanup(cell.lower())
    clean_cell = clean_cell.strip()
    lista_noticias_limpas += clean_cell
    lista_dados_target_0 += clean_cell.split()
    
for cell in dados_target_1:
    
    clean_cell = cleanup(cell.lower())
    clean_cell = clean_cell.strip()
    lista_noticias_limpas += clean_cell
    lista_dados_target_1 += clean_cell.split()
    
for cell in dados_target_2:
    
    clean_cell = cleanup(cell.lower())
    clean_cell = clean_cell.strip()
    lista_noticias_limpas += clean_cell
    lista_dados_target_2 += clean_cell.split()

In [17]:
# Conversão das listas para pd.Series
target_0 = pd.Series(lista_dados_target_0)
target_1 = pd.Series(lista_dados_target_1)
target_2 = pd.Series(lista_dados_target_2)

In [18]:
# Contagem de ocorrências:
tabela_0 = target_0.value_counts()
tabela_1 = target_1.value_counts()
tabela_2 = target_2.value_counts()

# Juntando as notícias
noticias_juntas = lista_dados_target_0 + lista_dados_target_1 + lista_dados_target_2

# Criando tabela com as notícias juntas e aplicando o value_counts
noticias_juntas_tabela = pd.Series(noticias_juntas)
noticias_juntas_tabela = noticias_juntas_tabela.value_counts()
noticias_juntas_tabela

de                   279
da                    87
a                     66
do                    57
em                    57
                    ... 
conspiracionistas      1
central                1
condenado              1
dois                   1
comprometem            1
Length: 1703, dtype: int64

In [19]:
# Começando os cálculos das probabilidades (prior)

prob_ser_0 = len(target_0)/len(noticias_juntas_tabela)

prob_ser_1 = len(target_1)/len(noticias_juntas_tabela)

prob_ser_2 = len(target_2)/len(noticias_juntas_tabela)

In [27]:
classificacao = []
for noticia in lista_noticias:
    prob_0 = 1
    prob_1 = 1
    prob_2 = 1

    # No for: probabilidade das palavras, descobre todas as probabilidades, aplica La Place, descobre qual a classificacao
    for palavra in noticia:
        
        #LA PLACE
        if palavra not in tabela_0:
            prob_0 *= 1/(len(target_0) + len(noticias_juntas_tabela)) 
        elif palavra in tabela_0:
            prob_0 *= (tabela_0[palavra] + 1)/(len(target_0) + len(noticias_juntas_tabela))
        
        
        if palavra not in tabela_1:
            prob_1 *= 1/(len(target_1) + len(noticias_juntas_tabela)) 
        elif palavra in tabela_1:
            prob_1 *= (tabela_1[palavra] + 1)/(len(target_1) + len(noticias_juntas_tabela))
        
        
        if palavra not in tabela_2:
            prob_2 *= 1/(len(target_2) + len(noticias_juntas_tabela)) 
        elif palavra in tabela_2:
            prob_2 *= (tabela_2[palavra] + 1)/(len(target_2) + len(noticias_juntas_tabela))
        
    
    # Cálculo das probabilidades
    prob_noticia_dado_0 = prob_0 * prob_ser_0
    prob_noticia_dado_1 = prob_1 * prob_ser_1
    prob_noticia_dado_2 = prob_2 * prob_ser_2
    

    
    #COMPARAÇÃO ENTRE OS RESULTADOS DE LA PLACE PARA CADA CATEGORIA
    if prob_noticia_dado_0 > prob_noticia_dado_1 and prob_noticia_dado_0 > prob_noticia_dado_2:
        classificacao.append(0)
            
    elif prob_noticia_dado_1 > prob_noticia_dado_0 and prob_noticia_dado_1 > prob_noticia_dado_2:
        #print(prob_1)
        classificacao.append(1)
            
    elif prob_noticia_dado_2 > prob_noticia_dado_0 and  prob_noticia_dado_2 > prob_noticia_dado_1:
        #print(prob_2)
        classificacao.append(2)

In [28]:
classificacao

[1, 1, 1, 1, 1, 1]

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [29]:
# Cria a lista a partir da planilha "Teste"
for noticia in test:
    clean_cell = cleanup(noticia.lower())
    lista_noticias.append(clean_cell.split())

In [30]:
# Comparando o resultado com a classificação feita manualmente

classificacao_final_serie = pd.Series(classificacao)
test['Classificação Final'] = classificacao_final_serie
test.tail(5)


,Categoria,Titulo,Descrição,Data,Pagina,Target,Classificação Final
394,tecnologia,Sony é processada em £ 5 bilhões por abuso de ...,"A Sony, gigante empresa de eletrônicos e jogos...",23/08/2022 08:50,5,2,NaN
395,tecnologia,Huawei lança Watch GT 2 Pro no Brasil; bateria...,Após o trazer os modelos de smartwatches GT2e ...,29/01/2021 16:43,232,1,NaN
396,tecnologia,J&J pede que regulador americano autorize vaci...,O laboratório Johnson & Johnson (J&J) solicito...,05/10/2021 16:37,95,0,NaN
397,tecnologia,Israel confirma primeiro caso de varíola do ma...,Israel registrou neste sábado (21) o primeiro ...,21/05/2022 15:05,26,0,NaN
398,tecnologia,Pesquisadores encontram novas alterações em li...,Pesquisadores da Fundação Oswaldo Cruz (Fiocru...,23/03/2021 14:33,196,0,NaN


In [31]:
# Tabela cruzada absoluta

pd.crosstab(test['Target'], test['Classificação Final'], margins=True) 

Classificação Final,1.0,All
Target,,
0,3,3
1,1,1
2,2,2
All,6,6


In [33]:
# Tabela cruzada relativa

pd.crosstab(test['Target'], test['Classificação Final'], margins=True) .round(4)*100

Classificação Final,1.0,All
Target,,
0,300,300
1,100,100
2,200,200
All,600,600


___
### Qualidade do Classificador a partir de novas separações das notícias entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas notícias. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas caterogias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por caterogia (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**